In [ ]:
#!pip install tweetpy

In [ ]:
import tweepy
import spacy
import pandas as pd
import json
import sys
import matplotlib.pyplot as plt
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

#sys.path.append("..\..\..")

In [ ]:
from config import api_key, secret_key, access_token, access_secret

In [ ]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(api_key, secret_key)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [ ]:
# Load model
nlp = spacy.load('en_core_web_md')
#accts = ("@BBC", "@CNN", "@nytimes", "@chrissyteigen", "@Minne_miraaa")

In [ ]:
def find(latest_tweet):
    myacct = "@butha1221"
    search_term = f"{myacct} analyze:"
    
    public_tweets = api.search(search_term, count=10, result_type="recent", since_id=latest_tweet)
    
    if len(public_tweets["statuses"]) > 0:
        tweet = public_tweets["statuses"]
        text = tweet["text"]
        tweeter = tweet["user"]["screen_name"]
        tweeter_at = f"@{tweeter}"
        tweet_id = tweet["id"]
        request = {"text": text,
                   "creator": tweeter_at,
                   "tweet ID": tweet_id}
        
        #split strings when accounts are found
        account_seperators = ':,;.!?'
        target_account = text.replace(search_term, "").strip(':,;.!?').strip()

        for seperator in account_seperators:
            if seperator in target_account:
                target_account = target_account.split(seperator)[0].strip()
    else:
        target_account = ''
        request = {}
        
    return target_account, request

In [ ]:
def most_recent(acct):
    tweets_data = []
    oldest_tweet = None
    
    for name in range(5):
        public_tweets = api.user_timeline(acct, count=100, result_type="recent", max_id=oldest_tweet)
        tweets_data.append(public_tweets)
        
        for tweet in public_tweets:
            tweet_id = tweet["id"]
            oldest_tweet = tweet_id - 1
            
    return tweets_data       

In [ ]:
def sentiment(sent):
    texts = []
    sentiments = []
    #former_tweets = []
    #compound_list = []
    #positive_list = []
    #negative_list = []
    #neutral_list = []
    
    counter = 0 
    
    for tweets in tweets_data:
        words = tweets["text"]
        results = analyzer.polarity_scores(words)
        compound = results["compound"]
        pos = results["pos"]
        neu = results["neu"]
        neg = results["neg"]
        
        former_tweets = counter
        texts.append(words)
        sentiments.append({"Compound": compound,
                           "Neutral": neu,
                           "Negative": neg,
                           "Positive": pos,
                           "Date": tweet["created_at"],
                           "Tweets Ago": former_tweets})
        counter = counter + 1
        
    sentiments_pd = pd.DataFrame.from_dict(sentiments)
    return sentiments_pd

In [ ]:
def plotit(sentiments_pd, target_account):
    plt.plot(sentiments_pd["Tweets Ago"], sentiments_pd["Compound"], label=target_account, marker=x)
    plot_name = f"polarity_{target_account}.png"
    plt.title("Sentiment Analysis of ")
    plt.xlabel("Tweets Ago")
    plt.ylabel("Polarity")
    plt.savefig(plot_name, bbox_inches="tight")
    plt.close()
    None
    
    return plot_name

In [ ]:
def send_tweet(plot_name, target_account, request):
    update = f"Tweet Sentiment Analysis: {target_account}"
    update_acct = api.update_status_with_media(filename=plot_name, status=update)
    
    return update_acct

In [ ]:
prevent_abuse = []
count = 0
newest_tweet = None

while count < 10:
    target_account, request = find(newest_tweet)
    if(target_account not in prevent_abuse):
        #newest_tweet = request[2] + 1
        prevent_abuse.append(target_account)
        
        try:
            tweets_data = most_recent(target_account)
            sentiments_pd = sentiments(tweets_data)
            plot_name = plotit(sentiments_pd, target_account)
            update_acct = send_tweet(plot_name, target_account, request)
            count = count + 1
            print(f"Success: {target_account} sentiment analysis complete!")
        except:
            print(f"{target_account} not found.")
    else:
        print("\nNo requests found.")
        
    time.sleep(60*5)